# Goals
- learn how to simulate beams in plasma
- get specra radiated by active CX between beam and plasma impurities
- get the beam attenuation
- show off with camera picture

# Contents
- Basic initiation and usage of cherab.core.Beam class
- Simulation of beam attenuation with cherab.core.model.SingleRayAttenuator
- Adding CX lines wiith cherab.core.model.BeamCXLine


In [ ]:
from cherab.compass.equilibrium.equilibrium import COMPASSEquilibrium
from raysect.optical import World
from cherab.openadas import OpenADAS
from cherab.core import Beam, Plasma, Species, Maxwellian
from cherab.core.atomic import elements, Line, deuterium, carbon, boron
from cherab.core.math import VectorAxisymmetricMapper, Interpolate1DCubic
from cherab.core.math.function import ScalarToVectorFunction2D
from raysect.core import Vector3D, Point3D, translate, rotate_basis
from cherab.core.model import SingleRayAttenuator, BeamCXLine
from raysect.optical.observer import PinholeCamera, FibreOptic

import numpy as np
from scipy.constants import atomic_mass, electron_mass
import matplotlib.pyplot as plt

get equilibrium obejct, generate plasma profiles and interpolators

In [ ]:
#Double parabolic function for plasma profile generation
def doubleparabola(r, Centre, Edge, p, q):
        return (Centre - Edge) * np.power((1 - np.power((r - r.min()) / (r.max() - r.min()), p)), q) + Edge

In [ ]:
#get equilibrium
path = "../data/efit_17636.h5"
shot_time = 1.125

# get equilibrium
equilibrium = COMPASSEquilibrium(path=path)
equilibrium_slice = equilibrium.time(shot_time)
psin_2d = equilibrium_slice.psi_normalised
#psin_3d = AxisymmetricMapper(equilibrium.psi_normalised)


In [ ]:
##### plasma profiles
psin_1d = np.linspace(0,1,100) #1d psi profile

te_profile = doubleparabola(psin_1d, 1000, 0, 2, 2) # 1d te profile
te_3d = equilibrium_slice.map3d((psin_1d, te_profile))

ne_profile = doubleparabola(psin_1d, 12e19, 0, 2, 2) # 1d ne profile
ne_3d = equilibrium_slice.map3d((psin_1d, ne_profile))

ti_profile = doubleparabola(psin_1d, 500, 0, 2, 2) # 1d ti profile
ti_3d = equilibrium_slice.map3d((psin_1d, ti_profile))

ni_profile = doubleparabola(psin_1d, 0.97*12e19, 0, 2, 2) # 1d ni profile
ni_3d = equilibrium_slice.map3d((psin_1d, ni_profile))

nc_profile = doubleparabola(psin_1d, 0.03*12e19, 0, 2, 2) # 1d nc6 profile
nc_3d = equilibrium_slice.map3d((psin_1d, nc_profile))

vy_profile = doubleparabola(psin_1d, -16e4, 0, 2, 2) # 1d velocity profile
vy = Interpolate1DCubic(psin_1d, vy_profile)

vi_3d = equilibrium_slice.map3d((psin_1d, vy_profile))

#function returning 3d velocity vector for cherab
flow_velocity = lambda x, y, z: Vector3D(y * vi_3d(x, y, z), - x * vi_3d(x, y, z), 0.) \
/ np.sqrt(x*x + y*y)

## setting up plasma 

In [ ]:
# setup scenegraph, plasma and populate it with particle species
world = World()

adas = OpenADAS(permit_extrapolation=True)

plasma = Plasma(parent=world)

d_distribution = Maxwellian(ni_3d, ti_3d, flow_velocity, deuterium.atomic_weight * atomic_mass)
c6_distribution = Maxwellian(nc_3d, ti_3d, flow_velocity, carbon.atomic_weight * atomic_mass)
b5_distribution = Maxwellian(nc_3d, ti_3d, flow_velocity, boron.atomic_weight * atomic_mass)

e_distribution = Maxwellian(ne_3d, te_3d, flow_velocity, electron_mass)

d_species = Species(deuterium, 1, d_distribution)
c6_species = Species(carbon, 6, c6_distribution)
b5_species = Species(boron, 5, b5_distribution)

plasma.electron_distribution = e_distribution
plasma.composition = [d_species, c6_species, b5_species]
plasma.b_field = VectorAxisymmetricMapper(equilibrium_slice.b_field)

# Setting up beams
- each beam component has to be a separate object 

In [ ]:
beam_radius = 1.2
beam_torangle = np.deg2rad(45)
integration_step = 0.01

beam_position = Point3D(-1.018, 1.81, 0)
beam_direction = Vector3D(1, -1, 0).normalise()
beams =[]
for comp, rat in zip([1, 2, 3], [0.8, 0.15, 0.05]):
    # setup scenegraph
    beam = Beam(parent=world, transform=translate(beam_position.x, beam_position.y, beam_position.z) *
                                        rotate_basis(beam_direction, Vector3D(0, 0, 1)))
    beam.plasma = plasma
    beam.atomic_data = adas
    beam.energy = 40000/comp
    beam.power = 35e4*rat
    beam.element = elements.deuterium
    beam.sigma = 0.02
    beam.divergence_x = 0.5
    beam.divergence_y = 0.5
    beam.length = 5.0
    beam.attenuator = SingleRayAttenuator(clamp_to_zero=True)
    beam.models = [
        BeamCXLine(Line(elements.deuterium, 0, (3, 2))),
        BeamCXLine(Line(elements.carbon, 5, (8, 7))),
        BeamCXLine(Line(elements.boron, 4, (7, 6))),
    ]
    beam.integrator.step = integration_step
    beam.integrator.min_samples = 5
    beams.append(beam)

# Beam Attenuation

In [ ]:
b0, b1, b2 = beams
b0_density = []
b1_density = []
b2_density = []
beam_distance = np.arange(1.5, 2.5, 0.01)
radius = []

for z in beam_distance:
    pnt = Point3D(0, 0, z).transform(beam.to(world))
    radius.append(np.sqrt(pnt.x**2 + pnt.y**2))
    b0_density.append(b0.attenuator.density(0, 0, z))
    b1_density.append(b1.attenuator.density(0, 0, z))
    b2_density.append(b2.attenuator.density(0, 0, z))

plt.figure()
plt.plot(beam_distance, b0_density, label='Full energy')
plt.plot(beam_distance, b1_density, label='Half energy')
plt.plot(beam_distance, b2_density, label='Third energy')
plt.legend()

plt.figure()
plt.plot(radius, b0_density, label='Full energy')
plt.plot(radius, b1_density, label='Half energy')
plt.plot(radius, b2_density, label='Third energy')
plt.legend()


# Simulation of CXRS diagnostic
- Added fibres observing beam along the path to get spectra evolution

In [ ]:
ferule_position = Point3D(0.565, 0.565, 0)

ferule = []
for i in np.linspace(1.5, 2, 5):
    spot = beam_position + beam_direction * i
    ferule.append(FibreOptic(acceptance_angle=1, radius=0.01, parent=world,
                   transform=translate(ferule_position.x,
                                       ferule_position.y,
                                       ferule_position.z) *
                             rotate_basis(ferule_position.vector_to(spot), Vector3D(0, 0, 1))))
    ferule[-1].min_wavelength = 490
    ferule[-1].max_wavelength = 500
    ferule[-1].spectral_bins = 1000
    ferule[-1].pixel_samples = 1000
    ferule[-1].observe()


# Camera Observation ov beam emission
How would a camera see the beam emission and its attenuation

In [ ]:
plt.ion()

camera = PinholeCamera((128, 128), parent=world, transform=translate(1, 1, 0)*
                                                           rotate_basis(Vector3D(-0.94, -0.34, 0), Vector3D(0, 0, 1)))
camera.spectral_rays = 1
camera.spectral_bins = 15
camera.pixel_samples = 100


In [ ]:
plt.ion()
camera.observe()